<a href="https://colab.research.google.com/github/haidour18/NLP/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install dependecies
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install pandas

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d https://www.kaggle.com/benhamner/nips-papers?select=papers.csv